# ----------------------------------------------定义模型--------------------------------------------------

- ### 分别定义编码器和解码器
- ### 编码器使用resnet152预训练模型
- ### 解码器使用LSTM模型

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn.utils.rnn import pack_padded_sequence

In [3]:
"""定义编码器"""
class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__() # 继承父类的属性定义
        resnet = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1) # 加载resnet152模型，并使用预训练权重
        modules = list(resnet.children())[:-1] # 将模型按列表的形式保存，并产出最后一行
        self.resnet = nn.Sequential(*modules) # 重新搭建模型
        self.linear = nn.Linear(resnet.fc.in_features, embed_size) # 创建线性输出层
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01) # 创建批量归一化层
        
    def forward(self, images):
        # 特征提取模式，不更新模型参数
        with torch.no_grad():
            features = self.resnet(images) # 不保留梯度直接进行特征提取
        features = features.reshape(features.size(0), -1) # 将输出调整为向量
        features = self.bn(self.linear(features)) # 对输出进行归一化
        return features 

In [4]:
"""定义解码器"""
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, max_seq_length=20):
        super(DecoderRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size) # 创建词嵌入层，将词汇表中的词元映射为指定维度的词嵌入向量
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        # 输入维度为embed_size,隐藏层维度为hidden_size，层数为num_layers，batch_first=true表示输入数据的第一个维度是批处理大小
        # hidden_size表示隐藏层大小，决定模型的记忆容量和表示能力，越大表示能力越好，但训练时间会变长
        # num_layers表示隐藏层堆叠数量，多层具有更强的表示能力，可以理解复杂的时序关系，但计算成本增加
        #  batch_first=True表示输入数据的维度，第一个维度是否为批量大小，
        # (batch_size, sequence_length, input_size)，这更符合一些数据处理习惯和库的要求
        self.linear = nn.Linear(hidden_size, vocab_size) # 将LSTM输出转换为词汇表大小的向量， 以便选择下一个单词
        self.max_seg_length = max_seq_length # 设置储存最大文本序列的长度，限制生成文本的长度
    
    def forward(self, features, captions, lengths): # features是图片的特征向量， captions是标注的词序列，lengths是序列的有效长度
        embeddings = self.embed(captions) # 将输入的标注进行词嵌入
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1) # 将特征向量和词嵌入拼接在一起，以便结合图片理解
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True) # 将拼接后的张量进行打包，以便处理变长序列，减少计算去除填充部分
        hiddens, _ = self.lstm(packed) # LSTM将学习文本序列的时序信息，并生成隐藏状态hiddens，其包含了文本序列的编码信息
        outputs = self.linear(hiddens[0]) # 将隐藏状态转为输出，表示每一个单词的得分，也就是每个单词的生成概率
        return outputs # 返回关于词汇表中每个单词的得分张量，可以用于生成图像描述文本
    
    def sample(self, features, states=None):
        # 使用贪心策略根据图片信息生成描述
        sampled_ids = [] # 创建空列表，储存生成词的索引
        inputs = features.unsqueeze(1) # 将features转化为与模型输入匹配的形状，也就是在第一维度上添加一个维度，表示时间步
        for i in range(self.max_seg_length): # 根据最大限制生成词
            hiddens, states = self.lstm(inputs, states) # 根据输入生成新的隐藏状态
            outputs = self.linear(hiddens.squeeze(1)) # 根据隐藏状态生成输出
            _, predicted = outputs.max(1) # 选择概率最高的词作为输出，返回其索引
            sampled_ids.append(predicted) # 将词的索引添加到空列表中
            inputs = self.embed(predicted) # 将生成的词转为词向量以便作为下一个预测的输入
            inputs = inputs.unsqueeze(1) # 调整词嵌入向量的形状，匹配下一个时间步的输入形状
        sampled_ids = torch.stack(sampled_ids, 1) # 根据索引将输出转换为词元
        return sampled_ids # 返回生成的话

# --------------------------------------测试代码是否可行--------------------------------------------

In [17]:
import argparse
import pickle
import import_ipynb
from Bulid_Vocab_2 import Vocabulary

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
"""创建一个命令行参数解析器对象，用于储存参数"""
parser = argparse.ArgumentParser() 

# 给定一些基础参数，包括文件路径等
parser.add_argument('--model_path', type=str, default='models/', help='保存训练模型的地方')
parser.add_argument('--crop_size', type=int, default=224, help='随机裁剪图片的大小')
parser.add_argument('--vocab_path', type=str, default='data/vocab.pkl', help='之前生成词典的路径')
parser.add_argument('--image_dir', type=str, default='data/resized2014', help='已经处理好大小的训练图片的路径')
parser.add_argument('--caption_path', type=str, default='data/annotations/captions_train2014.json',help='训练集标签的路径')
parser.add_argument('--log_step', type=int, default=10, help='打印训练进度的设定值')
parser.add_argument('--save_step', type=int, default=1000, help='保存模型节点的设定值')

# 设定模型的参数值
parser.add_argument('--embed_size', type=int, default=256, help='词嵌入向量的维度，也就是用多少维来表示一个词元')
parser.add_argument('--hidden_size', type=int, default=512, help='隐藏状态的维度')
parser.add_argument('--num_layers', type=int, default=1, help='LSTM层的数量')

# 设定训练的参数
parser.add_argument('--num_epochs', type=int, default=1, help='epoch数')
parser.add_argument('--batch_size', type=int, default=128, help='批量大小')
parser.add_argument('--num_workers', type=int, default=2, help='并行运算大小')
parser.add_argument('--learning_rate', type=float, default=0.001, help='学习率')

# 解析传入的命令行参数
args = parser.parse_args(args=[])
print(args)

Namespace(model_path='models/', crop_size=224, vocab_path='data/vocab.pkl', image_dir='data/resized2014', caption_path='data/annotations/captions_train2014.json', log_step=10, save_step=1000, embed_size=256, hidden_size=512, num_layers=1, num_epochs=1, batch_size=128, num_workers=2, learning_rate=0.001)


In [20]:
"""指定 vocab.pkl词表 文件的路径"""
file_path = 'data/vocab.pkl'

#使用 pickle 模块打开文件
with open(file_path, 'rb') as file:
     vocab = pickle.load(file)

In [21]:
encoder = EncoderCNN(args.embed_size).to(device)
decoder = DecoderRNN(args.embed_size, args.hidden_size, len(vocab), args.num_layers).to(device)

In [22]:
print(encoder)

EncoderCNN(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64

In [23]:
print(decoder)

DecoderRNN(
  (embed): Embedding(9948, 256)
  (lstm): LSTM(256, 512, batch_first=True)
  (linear): Linear(in_features=512, out_features=9948, bias=True)
)
